# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy.stats import percentileofscore as score #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2171330624776,
 'week52high': 138.26,
 'week52low': 56.25,
 'week52change': 0.6848473308833685,
 'sharesOutstanding': 17168007397,
 'float': 0,
 'avg10Volume': 119146234,
 'avg30Volume': 119802673,
 'day200MovingAvg': 118.63,
 'day50MovingAvg': 130.88,
 'employees': 0,
 'ttmEPS': 3.32,
 'ttmDividendRate': 0.8171750832094088,
 'dividendYield': 0.006537795443899875,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-23',
 'nextEarningsDate': '0',
 'peRatio': 39.3537136060109,
 'beta': 1.2032146705326727,
 'maxChangePercent': 50.04483190644727,
 'year5ChangePercent': 4.751017356399992,
 'year2ChangePercent': 2.504410064283661,
 'year1ChangePercent': 0.6779428307337718,
 'ytdChangePercent': -0.0301500245625052,
 'month6ChangePercent': 0.3489184267389703,
 'month3ChangePercent': 0.0375423440126267,
 'month1ChangePercent': 0.05435373269958135,
 'day30ChangePercent': 0.05352275894514232,
 'day5ChangePercent': -0.01709771819176434}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.6779428307337718

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [20]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['price'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.401,0.477569,N/A
1,AAL,15.818,-0.452556,N/A
2,AAP,180.880,0.203525,N/A
3,AAPL,133.177,0.681544,N/A
4,ABBV,117.320,0.301573,N/A
...,...,...,...,...
500,YUM,110.967,0.0825539,N/A
501,ZBH,164.724,0.0774002,N/A
502,ZBRA,418.802,0.659385,N/A
503,ZION,50.130,0.0320295,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [21]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.420,2.53658,N/A
1,ALB,183.060,1.63986,N/A
2,LB,47.350,1.46693,N/A
3,FCX,30.960,1.42058,N/A
4,NVDA,547.350,1.27079,N/A
5,PYPL,256.340,1.14964,N/A
6,WST,314.394,1.00074,N/A
7,AMD,91.880,0.991678,N/A
8,ALGN,567.440,0.89213,N/A
9,PWR,76.660,0.876518,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [22]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [23]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

C:\Users\PLT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.420,2.53658,4733
1,ALB,183.060,1.63986,1071
2,LB,47.350,1.46693,4141
3,FCX,30.960,1.42058,6333
4,NVDA,547.350,1.27079,358
5,PYPL,256.340,1.14964,764
6,WST,314.394,1.00074,623
7,AMD,91.880,0.991678,2134
8,ALGN,567.440,0.89213,345
9,PWR,76.660,0.876518,2557


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [24]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.149,N/A,0.476243,N/A,0.440679,N/A,0.216415,N/A,0.0747729,N/A,N/A
1,AAL,15.290,N/A,-0.446056,N/A,0.301395,N/A,0.194432,N/A,-0.0990804,N/A,N/A
2,AAP,181.720,N/A,0.199229,N/A,0.328092,N/A,0.134845,N/A,0.126432,N/A,N/A
3,AAPL,134.586,N/A,0.685038,N/A,0.358664,N/A,0.03897,N/A,0.0530081,N/A,N/A
4,ABBV,115.060,N/A,0.301914,N/A,0.164451,N/A,0.262734,N/A,0.0334401,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.967,N/A,0.0792192,N/A,0.251468,N/A,0.128819,N/A,0.0164417,N/A,N/A
501,ZBH,164.260,N/A,0.0749716,N/A,0.347853,N/A,0.0757478,N/A,0.132634,N/A,N/A
502,ZBRA,419.040,N/A,0.683099,N/A,0.582306,N/A,0.429898,N/A,0.0831978,N/A,N/A
503,ZION,49.740,N/A,0.0323103,N/A,0.598447,N/A,0.597993,N/A,0.208866,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [41]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
            
#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.149,N/A,0.476243,87.9208,0.440679,70.6931,0.216415,65.5446,0.0747729,62.5743,N/A
1,AAL,15.290,N/A,-0.446056,1.38614,0.301395,54.6535,0.194432,61.9802,-0.0990804,0.990099,N/A
2,AAP,181.720,N/A,0.199229,61.9802,0.328092,57.4257,0.134845,52.0792,0.126432,80.396,N/A
3,AAPL,134.586,N/A,0.685038,94.8515,0.358664,62.1782,0.03897,32.6733,0.0530081,53.4653,N/A
4,ABBV,115.060,N/A,0.301914,74.8515,0.164451,33.0693,0.262734,72.0792,0.0334401,41.9802,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.967,N/A,0.0792192,44.9505,0.251468,46.1386,0.128819,50.6931,0.0164417,34.2574,N/A
501,ZBH,164.260,N/A,0.0749716,43.9604,0.347853,60.198,0.0757478,39.802,0.132634,83.1683,N/A
502,ZBRA,419.040,N/A,0.683099,94.6535,0.582306,85.1485,0.429898,87.5248,0.0831978,66.3366,N/A
503,ZION,49.740,N/A,0.0323103,37.4257,0.598447,86.3366,0.597993,95.2475,0.208866,95.8416,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [42]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.149,N/A,0.476243,87.9208,0.440679,70.6931,0.216415,65.5446,0.0747729,62.5743,71.6832
1,AAL,15.290,N/A,-0.446056,1.38614,0.301395,54.6535,0.194432,61.9802,-0.0990804,0.990099,29.7525
2,AAP,181.720,N/A,0.199229,61.9802,0.328092,57.4257,0.134845,52.0792,0.126432,80.396,62.9703
3,AAPL,134.586,N/A,0.685038,94.8515,0.358664,62.1782,0.03897,32.6733,0.0530081,53.4653,60.7921
4,ABBV,115.060,N/A,0.301914,74.8515,0.164451,33.0693,0.262734,72.0792,0.0334401,41.9802,55.495
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.967,N/A,0.0792192,44.9505,0.251468,46.1386,0.128819,50.6931,0.0164417,34.2574,44.0099
501,ZBH,164.260,N/A,0.0749716,43.9604,0.347853,60.198,0.0757478,39.802,0.132634,83.1683,56.7822
502,ZBRA,419.040,N/A,0.683099,94.6535,0.582306,85.1485,0.429898,87.5248,0.0831978,66.3366,83.4158
503,ZION,49.740,N/A,0.0323103,37.4257,0.598447,86.3366,0.597993,95.2475,0.208866,95.8416,78.7129


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [43]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [44]:
portfolio_input()

Enter the value of your portfolio:10000000


In [45]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\PLT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.149,1506,0.476243,87.9208,0.440679,70.6931,0.216415,65.5446,0.0747729,62.5743,71.6832
1,AAL,15.290,12823,-0.446056,1.38614,0.301395,54.6535,0.194432,61.9802,-0.0990804,0.990099,29.7525
2,AAP,181.720,1079,0.199229,61.9802,0.328092,57.4257,0.134845,52.0792,0.126432,80.396,62.9703
3,AAPL,134.586,1456,0.685038,94.8515,0.358664,62.1782,0.03897,32.6733,0.0530081,53.4653,60.7921
4,ABBV,115.060,1704,0.301914,74.8515,0.164451,33.0693,0.262734,72.0792,0.0334401,41.9802,55.495
5,ABC,110.728,1770,0.254402,68.9109,0.110936,23.3663,0.106822,45.7426,0.0901861,69.703,51.9307
6,ABMD,332.068,590,0.742177,95.8416,0.233422,43.3663,0.171009,57.8218,0.25711,97.8218,73.7129
7,ABT,115.000,1705,0.306692,76.2376,0.181468,36.6337,-0.0173092,17.8218,0.0177521,35.0495,41.4356
8,ACN,263.960,742,0.267303,70.099,0.187427,37.8218,0.123369,49.1089,0.052694,52.8713,52.4752
9,ADBE,488.158,401,0.395021,83.3663,0.0122093,12.8713,-0.0771275,9.30693,-0.00929546,21.5842,31.7822


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [46]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [47]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [48]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [49]:
writer.save()